In [1]:
import geopandas as gpd
import pandas as pd
import pandana as pdn
import osmnx as ox
import numpy as np
import warnings

# Desactivar todas las advertencias
warnings.filterwarnings("ignore")


def get_path(source ,feature, mode='caminata'):
    return f'/app/data/{source}/red_escenario_futuro/{mode}/{feature}_conce'
def get_input_nodes():
    return get_path('input', 'nodes')
def get_input_edges():
    return get_path('input', 'edges')
def get_output_nodes():
    return get_path('output', 'nodes')
def get_output_edges():
    return get_path('output', 'edges')

In [2]:
nodes_df = gpd.read_file(get_output_nodes())
edges_df = gpd.read_file(get_output_edges())

In [3]:
nodes = pd.DataFrame(
    {
        'osmid': nodes_df['osmid'].astype(int),
        'lat' : nodes_df.geometry.y.astype(float),
        'lon' : nodes_df.geometry.x.astype(float),
        'y' : nodes_df.geometry.y.astype(float),
        'x' : nodes_df.geometry.x.astype(float),
    }
)
nodes['id'] = nodes['osmid'].values

nodes = gpd.GeoDataFrame(data=nodes, geometry=nodes_df.geometry)
nodes.set_index('osmid', inplace=True)

In [4]:
edges = pd.DataFrame(
    {
        'u': edges_df['u'].astype(int),
        'v': edges_df['v'].astype(int),
        'from': edges_df['u'].astype(int),
        'to': edges_df['v'].astype(int),
        'osmid': edges_df['osmid'].astype(int),
        'length': edges_df['length'].astype(float)
    }
)
edges['key'] = 0
edges['key'] = edges['key'].astype(int)
edges = gpd.GeoDataFrame(data=edges, geometry=edges_df.geometry)
edges.set_index(['u', 'v', 'key'], inplace=True)

In [13]:
net = pdn.Network(
    nodes['lon'],
    nodes['lat'],
    edges['from'],
    edges['to'],
    edges[['length']]
)

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 36958
Setting CH edge vector of size 81667
Range graph removed 82002 edges of 163334
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [6]:
node_ids = list(net.node_ids)

In [43]:
total_nodes = len(node_ids)
dict_lenghts = {}

for n in range(total_nodes):
    actual_node = node_ids[n]

    nodes_destination = node_ids[n:]
    node_source = [actual_node]*len(nodes_destination)
    lengths = net.shortest_path_lengths(node_source, nodes_destination)


    dict_lenghts[actual_node] = {node: length for node, length in zip(nodes_destination, lengths)}


In [54]:
routing_matrix = pd.DataFrame.from_dict(dict_lenghts)
routing_matrix = routing_matrix.transpose()
routing_matrix.to_pickle('/app/data/output/routing_matrix.pkl')

,267252253,267252256,267252355,267548089,267548092,267548147,267548208,267548275,267548291,267548299
267252253,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267252256,4.818,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267252355,321.660,325.294,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267548089,4635.995,4639.629,4338.684,0.000,NaN,NaN,NaN,NaN,NaN,NaN
267548092,4266.622,4270.256,3969.311,369.373,0.000,NaN,NaN,NaN,NaN,NaN
267548147,912.595,917.413,1187.107,3956.227,3586.854,0.000,NaN,NaN,NaN,NaN
267548208,3865.425,3869.059,3568.114,1815.013,1445.640,3537.420,0.000,NaN,NaN,NaN
267548275,856.251,861.069,1130.763,3991.159,3621.786,109.014,3532.846,0.000,NaN,NaN
267548291,873.807,878.625,1051.908,3851.248,3481.875,199.303,3389.887,142.959,0.000,NaN
267548299,1793.027,1796.661,1495.716,3094.249,2724.876,1375.074,2162.346,1370.500,1227.541,0.0
